1. Azure Kinect SDK와 Azure Kinect Body Tracking SDK를 설치한다.
 - https://learn.microsoft.com/ko-kr/azure/kinect-dk/sensor-sdk-download
 - https://learn.microsoft.com/en-us/azure/kinect-dk/body-sdk-download

2. pykinect_azure를 설치한다.

3. 참고
 - 관절별 인덱스 : https://learn.microsoft.com/ko-kr/azure/kinect-dk/body-joints
 - 팔꿈치 토크 수식 : (몸무게/16 * 2 + 물건무게) * (키/6) * sin(각도)
 - 어깨 토크 수식 : {키/6 * (몸무게/16 * 2) * sin(각도)} + {키/3 * 든물건무게 * sin(각도)}

In [ ]:
import sys
import cv2
import numpy as np
import pykinect_azure as pykinect

# 패키지에서 출력하는 로그를 숨기기 위한 코드
stdout = sys.stdout
sys.stdout = None

# 출력을 복구하고 싶다면 아래 코드로 가능
# sys.stdout = stdout

# Initialize the library, if the library is not found, add the library path as argument
pykinect.initialize_libraries(track_body=True)

# Modify camera configuration
device_config = pykinect.default_configuration
device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1080P
device_config.depth_mode = pykinect.K4A_DEPTH_MODE_WFOV_2X2BINNED
#print(device_config)

# Start device
device = pykinect.start_device(config=device_config)

# Start body tracker
bodyTracker = pykinect.start_body_tracker()

while True:
    
    # Get capture
    capture = device.update()

    try:
        ret, color_image = capture.get_color_image()
        body_frame = bodyTracker.update()
    except:
        continue

    if not ret:
        break

    # Draw the skeletons into the color image
    color_skeleton = body_frame.draw_bodies(color_image, pykinect.K4A_CALIBRATION_TYPE_COLOR)
    info = body_frame.get_bodies()[0].json()

    neck = info["skeleton"]["joints"][3]
    pelvis = info["skeleton"]["joints"][0]
    hip_left = info["skeleton"]["joints"][18]
    hip_right = info["skeleton"]["joints"][22]
    elbow_left = info["skeleton"]["joints"][6]
    wrist_left = info["skeleton"]["joints"][7]

    #허리길이 측정을 위해 두 벡터 간 유클리드 거리 계산
    vac_neck = np.array(neck["position"]["v"])
    vac_pelvis = np.array(pelvis["position"]["v"])
    dist = np.linalg.norm(vac_neck-vac_pelvis)

    dst = cv2.putText(color_skeleton, str(round(dist, 2)), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    dst = cv2.resize(dst, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    cv2.imshow("dst", dst)

    sys.stdout = stdout
    print(elbow_left["orientation"]["v"], end="\r")
    sys.stdout = None

    if cv2.waitKey(1) == 27:
        sys.stdout = stdout
        break

cv2.destroyAllWindows()
device.close()

In [ ]:
device.close()

In [ ]:
elbow_left